<a href="https://colab.research.google.com/github/kahuroA/SQL-PRACTICE/blob/main/SQL_MINI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SQL MINI-PROJECT** *

In [1]:
#load the sql extension
%load_ext sql
#create a temporary connection of sqlite
%sql sqlite://

'Connected: @None'

In [2]:
#import packages and read pandas dataframe in an sql table
import pandas as pd
with open('GrandElectors_by_state.csv','r') as f:
  Grand_Electors=pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE IF EXISTS Grand_Electors;
%sql PERSIST Grand_Electors;
%sql SELECT * FROM Grand_Electors LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [3]:
#open the population csv with pandas and store and read the dataframe using sql table
with open('Population_by_state.csv','r') as f:
  Population=pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE IF EXISTS Population;
%sql PERSIST Population;
%sql SELECT * FROM Population LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [4]:
# capitalize the states in the Grand_Electors table so as to enable an inner join
%sql UPDATE Grand_Electors SET State=UPPER(State);

 * sqlite://
51 rows affected.


[]

In [5]:
%%sql
--Make the inner join between Grand_Electors and Population, store the result in a new table called Combined
CREATE TABLE Combined AS
SELECT * FROM Grand_Electors
INNER JOIN Population  on Population.State = Grand_Electors.State;

 * sqlite://
Done.


[]

In [6]:
%%sql
--view the combined table
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,State:1,Population
ALABAMA,9,ALABAMA,4858979
ALASKA,3,ALASKA,738432
ARIZONA,11,ARIZONA,6828065
ARKANSAS,6,ARKANSAS,2978204
CALIFORNIA,55,CALIFORNIA,39144818


In [7]:
%%sql
--We will drop the redundant column State:1 by creating a table Combined_new with relevant columns from Combined.
CREATE TABLE Combined_new (
  State  TEXT PRIMARY KEY,
  GrandElectors  int,
  Population  int
);
--After creating the table with relevant columns from Combine table, we will insert columns from Combined table 
INSERT INTO Combined_new SELECT State, GrandElectors, Population FROM Combined;
--We will then drop the table Combined
DROP TABLE Combined;
--finally we will alter the new table's name and give it the name Combined
ALTER TABLE Combined_new RENAME TO Combined;

 * sqlite://
Done.
Done.
Done.
Done.


[]

In [8]:
%%sql
--view the table combined
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818


2. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.

In [9]:
%%sql
--We update the state column with a condition of where we need to do the update
UPDATE Combined
SET State='DC'
WHERE State='DISTRICT OF COLUMBIA'

 * sqlite://
Done.


[]

3. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.

In [10]:
%%sql
--create a new table with the ratio column
CREATE TABLE Combined_new AS
SELECT *, CAST(Combined.GrandElectors AS FLOAT)/CAST(Combined.Population AS FLOAT) AS Ratio
FROM   Combined;
--drop the old table Combined
DROP TABLE Combined;
--Rename the new table with ratio column as Combined
ALTER TABLE Combined_new Rename to Combined;

 * sqlite://
Done.
Done.
Done.


[]

In [11]:
%%sql
--Let us view the table with the new ratio column
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
ALABAMA,9,4858979,1.8522409749043987e-06
ALASKA,3,738432,4.0626625065002604e-06
ARIZONA,11,6828065,1.610998137832607e-06
ARKANSAS,6,2978204,2.0146370094190997e-06
CALIFORNIA,55,39144818,1.4050391037710278e-06


4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list

In [12]:
%%sql
CREATE TABLE Sorted AS
SELECT * FROM Combined
ORDER BY Ratio DESC;

 * sqlite://
Done.


[]

In [13]:
%%sql
--let us view the new table with ratio sorted in decreasing order
SELECT * FROM Sorted LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
WYOMING,3,586107,5.118519314732634e-06
VERMONT,3,626042,4.792010759661493e-06
DC,3,672228,4.462771559649405e-06
ALASKA,3,738432,4.0626625065002604e-06
NORTH DAKOTA,3,756927,3.963394092164766e-06


# EXPLORATION AND SUMMARY STATISTICS IN THE FULL LIST

In [38]:
#Find the state with highest number of Grand Electors 
%%sql
SELECT State,GrandElectors FROM Sorted WHERE GrandElectors= (SELECT MAX(GrandElectors) FROM Sorted);

 * sqlite://
Done.


State,GrandElectors
CALIFORNIA,55


In [39]:
#Find the states with the lowest number of Grand Electors
%%sql
SELECT State,GrandElectors FROM Sorted WHERE GrandElectors= (SELECT MIN(GrandElectors) FROM Sorted);

 * sqlite://
Done.


State,GrandElectors
WYOMING,3
VERMONT,3
DC,3
ALASKA,3
NORTH DAKOTA,3
SOUTH DAKOTA,3
DELAWARE,3
MONTANA,3


In [40]:
#Find the state with the least population
%%sql
SELECT State,Population FROM Sorted WHERE Population= (SELECT MIN(Population) FROM Sorted);

 * sqlite://
Done.


State,Population
WYOMING,586107


In [41]:
#Find the state with the highest population
%%sql
SELECT State,Population FROM Sorted WHERE Population= (SELECT MAX(Population) FROM Sorted);

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818


In [42]:
#Find the average number of grand electors
%%sql
SELECT AVG(GrandElectors) FROM Sorted;

 * sqlite://
Done.


AVG(GrandElectors)
10.549019607843137


In [43]:
#Find the average population in US
%%sql
SELECT AVG(Population) FROM Sorted;

 * sqlite://
Done.


AVG(Population)
6302329.803921568


In [44]:
#Find the states with population greater than the average
%%sql
SELECT State, Population FROM Sorted
WHERE Population>=(SELECT AVG(Population) FROM Sorted);

 * sqlite://
Done.


State,Population
WASHINGTON,7170351
TENNESSEE,6600299
INDIANA,6619680
MASSACHUSETTS,6794422
MICHIGAN,9922576
ARIZONA,6828065
GEORGIA,10214860
NEW JERSEY,8958013
PENNSYLVANIA,12802503
ILLINOIS,12859995


**5. To compute the running total of Grand Electors in that sorted list**

In [14]:
%%sql
CREATE TABLE Sorted_Cummulative AS
SELECT
    t.State,t.GrandElectors,t.Population,t.Ratio,
    (SELECT Sum(r.GrandElectors) FROM Sorted as r WHERE t.Ratio <= r.Ratio) AS Cummulative
FROM Sorted as t
Group By t.Ratio ORDER BY t.Ratio DESC;

 * sqlite://
Done.


[]

In [64]:
%%sql
--Let us view the first 5 columns of the new table with the cummulative sum.
SELECT * FROM Sorted_Cummulative LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15


**6. Independently, to compute the half of the total of Grand Electors overall (in the whole country)**

In [16]:
%%sql
SELECT SUM(GrandElectors)/2 FROM Sorted_Cummulative;

 * sqlite://
Done.


SUM(GrandElectors)/2
269


**7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.**

In [26]:
%%sql
CREATE TABLE Target_List AS
SELECT * FROM Sorted_Cummulative WHERE Cummulative<=269;

 * sqlite://
Done.


[]

In [27]:
%%sql
--Let us view our target list
SELECT * FROM Target_List

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15
RHODE ISLAND,4,1056298,3.7868101615263876e-06,19
SOUTH DAKOTA,3,858469,3.4945932817609025e-06,22
DELAWARE,3,945934,3.1714686225466047e-06,25
MAINE,4,1329328,3.009039153617467e-06,29
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06,33


**a. Add the first state for which the running total is larger than the threshold.**

In [28]:
%%sql
--Let us add the first state for which the running total is larger than the threshold
INSERT INTO Target_List(State, GrandElectors, Population, Ratio, Cummulative)
SELECT * FROM Sorted_Cummulative WHERE Cummulative>269 LIMIT 1;

 * sqlite://
Done.


[]

In [29]:
%%sql
--Let us view the Target List with the additional row
SELECT * FROM Target_List

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15
RHODE ISLAND,4,1056298,3.7868101615263876e-06,19
SOUTH DAKOTA,3,858469,3.4945932817609025e-06,22
DELAWARE,3,945934,3.1714686225466047e-06,25
MAINE,4,1329328,3.009039153617467e-06,29
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06,33


# QUESTIONS

1. How many states do you end up within the target list?

In [30]:
%%sql
SELECT COUNT(State) FROM Target_List;

 * sqlite://
Done.


COUNT(State)
40


2. Is it a small or large number?  It is a large number since it is 78% of the total states in the US

In [63]:
Total_states_Target_list=40
Total_states_US=51
(Total_states_Target_list/Total_states_US)*100

78.43137254901961

3.Do you think it would be a good recommendation to target those states?

# SUMMARY STATISTICS OF TARGET LIST

In [49]:
#State with the maximum grand electors in the target list
%%sql
SELECT State,GrandElectors FROM Target_List WHERE GrandElectors= (SELECT MAX(GrandElectors) FROM Target_List);

 * sqlite://
Done.


State,GrandElectors
MICHIGAN,16


In [50]:
#States with the minimum grand electors in the target list
%%sql
SELECT State,GrandElectors FROM Target_List WHERE GrandElectors= (SELECT MIN(GrandElectors) FROM Target_List);

 * sqlite://
Done.


State,GrandElectors
WYOMING,3
VERMONT,3
DC,3
ALASKA,3
NORTH DAKOTA,3
SOUTH DAKOTA,3
DELAWARE,3
MONTANA,3


In [51]:
#State with the least population in the target list
%%sql
SELECT State,Population FROM Target_List WHERE Population= (SELECT MIN(Population) FROM Target_List);

 * sqlite://
Done.


State,Population
WYOMING,586107


In [52]:
#State with the largest population in the target list
%%sql
SELECT State,Population FROM Target_List WHERE Population= (SELECT MAX(Population) FROM Target_List);

 * sqlite://
Done.


State,Population
MICHIGAN,9922576


In [53]:
#Average number of grand electors in the target list
%%sql
SELECT AVG(GrandElectors) FROM Target_List;

 * sqlite://
Done.


AVG(GrandElectors)
6.775


In [57]:
#Median Grand Electors in the target list
%%sql
SELECT GrandElectors
FROM Target_List
ORDER BY GrandElectors
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM Target_List) / 2

 * sqlite://
Done.


GrandElectors
6


In [58]:
#Mode gtand electors in the target list
%%sql
SELECT GrandElectors,COUNT(GrandElectors) AS Frequency
FROM Target_List
GROUP BY GrandElectors
ORDER BY Frequency Desc LIMIT 1;

 * sqlite://
Done.


GrandElectors,Frequency
3,8


In [54]:
#Average population of states in the target list
%%sql
SELECT AVG(Population) FROM Target_List;

 * sqlite://
Done.


AVG(Population)
3496580.9


In [59]:
#median population in the target list
%%sql
SELECT Population
FROM Target_List
ORDER BY Population
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM Target_List) / 2

 * sqlite://
Done.


Population
2995919


In [55]:
#States with population greater than average in the target list
%%sql
SELECT State, Population FROM Target_List
WHERE Population>=(SELECT AVG(Population) FROM Target_List);

 * sqlite://
Done.


State,Population
CONNECTICUT,3590886
ALABAMA,4858979
SOUTH CAROLINA,4896146
MINNESOTA,5489594
KENTUCKY,4425092
OKLAHOMA,3911338
OREGON,4028977
WISCONSIN,5771337
LOUISIANA,4670724
WASHINGTON,7170351
